In [ ]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
from tensorflow.python.ops import lookup_ops

import numpy as np

In [ ]:
tf.enable_eager_execution()

In [ ]:
tf.set_random_seed(42)

In [ ]:
from rnn_lm import LanguageModel, create_dataset, loss_fun

In [ ]:
train_file = 'simple-examples/data/ptb.train.txt'
valid_file = 'simple-examples/data/ptb.valid.txt'
vocab_file = 'simple-examples/data/vocab.txt'

In [ ]:
vocab_table = lookup_ops.index_table_from_file(vocab_file)

In [ ]:
#Specify model params
V = int(vocab_table.size())
cell = 'lstm'
d = 128
h = 128

BATCH_SIZE=32

In [ ]:
lm = LanguageModel(V=V, cell=cell, d=d, h=h)

In [ ]:
train_dataset = create_dataset(train_file, batch_size=BATCH_SIZE, vocab_table=vocab_table)

In [ ]:
datum = next(iter(train_dataset))

In [ ]:
loss_fun(lm, datum)

In [ ]:
checkpoint_dir = 'lm'

In [ ]:
opt = tf.train.AdamOptimizer(learning_rate=0.001)

In [ ]:
root = tfe.Checkpoint(optimizer=opt, model=lm, optimizer_step=tf.train.get_or_create_global_step())

In [ ]:
root.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
tf.train.get_or_create_global_step()

In [ ]:
loss_fun(lm, datum)

### Resume training!

In [ ]:
loss_and_grads_fun = tfe.implicit_value_and_gradients(loss_fun)

In [ ]:
NUM_EPOCHS = 10
STATS_STEPS = 50
for epoch_num in range(NUM_EPOCHS):
    batch_loss = []
    for step_num, datum in enumerate(train_dataset, start=1):
        loss_value, gradients = loss_and_grads_fun(lm, datum)
        batch_loss.append(loss_value)
        
        if step_num % STATS_STEPS == 0:
            print(f'Epoch: {epoch_num} Step: {step_num} Avg Loss: {np.average(np.asarray(loss_value))}')
            batch_loss = []
        opt.apply_gradients(gradients, global_step=tf.train.get_or_create_global_step())
    print(f'Epoch{epoch_num} Done!')